Detección de caras con webcam

In [1]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [21]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 0
imodoE = 0

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    # Face detector change
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    #Eye detector change
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0


Camera 0


## TAREA

In [143]:
import cv2
import FaceNormalizationUtils as faceutils
import FaceDetectors

def combine_images(frame, image, alpha, imagepos, dim):
    roi = frame[imagepos[1]:imagepos[1]+dim[1],imagepos[0]:imagepos[0]+dim[0]]
    imageroi = image[:,:,0:3]
    inversemask = cv2.bitwise_not(alpha)
    maskedimageroi = cv2.bitwise_and(imageroi, imageroi, mask=alpha)
    maskedroi = cv2.bitwise_and(roi, roi, mask=inversemask)
    fused = cv2.add(maskedroi, maskedimageroi)
    return fused

normalizatorHS = faceutils.Normalization()

vid = cv2.VideoCapture(0)

if not vid.isOpened():
    vid = cv2.VideoCapture(1)
    if not cap.isOpened():
        vid = cv2.VideoCapture(0)
        if not vid.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

vid.set(3,640);
vid.set(4,480);


while True:
    ret, frame = vid.read()
    # for HS normalization
    B, G, R = cv2.split(frame)

    # search face
    values_1 = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
    values_0 = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[0])
    if values_1 is not None and values_0 is not None:
        face_, eyes_, shape_ = values_0
        face, eyes, shape = values_1

        # face container
        [xf, yf , wf, hf] = face

        d_ = (int(wf/2), int(wf/2))
        dn_ = (int(wf/2), int(wf/4))
        leftear = cv2.imread('./images/left_ear.png', cv2.IMREAD_UNCHANGED)
        rightear = cv2.imread('./images/right_ear.png', cv2.IMREAD_UNCHANGED)
        nose = cv2.imread('./images/nose.png', cv2.IMREAD_UNCHANGED)

        if xf > -1:
            # eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                leftear = cv2.resize(leftear, d_)
                rightear = cv2.resize(rightear, d_)
                nose = cv2.resize(nose, dn_)

                leftear_ = (shape[17][0]-int(d_[0]/2), shape[17][1]-int(d_[1]))
                rightear_ = (shape[26][0]-int(d_[0]/2), shape[26][1]-int(d_[1]))
                nose_ = (shape_[4][0]-int(dn_[0]/2), shape_[4][1]-int(dn_[1]))

                fusedleft = combine_images(frame, leftear, leftear[:,:,3], leftear_, d_)
                fusedright = combine_images(frame, rightear, rightear[:,:,3], rightear_, d_)
                fusednose = combine_images(frame, nose, nose[:,:,3], nose_, dn_)

                frame[leftear_[1]:leftear_[1]+d_[1], leftear_[0]:leftear_[0]+d_[0]] = fusedleft
                frame[rightear_[1]:rightear_[1]+d_[1], rightear_[0]:rightear_[0]+d_[0]] = fusedright
                frame[nose_[1]:nose_[1]+dn_[1], nose_[0]:nose_[0]+dn_[0]] = fusednose


    cv2.imshow('Cam', frame)

    # stop with esc key
    if cv2.waitKey(20) == 27:
        break

# close windows and release camera
vid.release()
cv2.destroyAllWindows()

Camera 0
